# Akıllı Çöp Kutusu Veri Analizi

Bu notebook, akıllı çöp kutularından toplanan verilerin analiz edilmesini içermektedir.

Amaç: Konteyner türü ve atık türü kombinasyonlarının doluluk oranlarını analiz etmek.

## 1. Kutuphanelerin Yuklenmesi

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 2. Veri Yukleme ve Inceleme

In [ ]:
df = pd.read_csv("Smart_Bin.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

## 3. Kategorik Degiskenlerin Incelenmesi

In [ ]:
df['Container Type'].unique()

In [ ]:
df['Recyclable fraction'].unique()

In [ ]:
print("Konteyner Türü Dağılımı:")
print(df['Container Type'].value_counts())
print("\nAtık Türü Dağılımı:")
print(df['Recyclable fraction'].value_counts())

## 4. Pivot Tablo Analizi

Konteyner turu ve atik turu kombinasyonlarinin ortalama doluluk oranlarini hesapliyoruz.

In [ ]:
pivot_fl_b = pd.pivot_table(df, values='FL_B', index='Container Type', 
                            columns='Recyclable fraction', aggfunc='mean')
pivot_fl_b = pivot_fl_b.round(2)
pivot_fl_b

In [ ]:
pivot_fl_a = pd.pivot_table(df, values='FL_A', index='Container Type', 
                            columns='Recyclable fraction', aggfunc='mean')
pivot_fl_a = pivot_fl_a.round(2)
pivot_fl_a

In [ ]:
pivot_degisim = pivot_fl_b - pivot_fl_a
pivot_degisim = pivot_degisim.round(2)
pivot_degisim

## 5. Gruplama Analizleri

In [ ]:
konteyner_ort = df.groupby('Container Type')['FL_B'].mean().sort_values(ascending=False)
konteyner_ort.round(2)

In [ ]:
sinif_analiz = df.groupby('Class').agg({
    'FL_B': ['mean', 'std', 'count'],
    'FL_A': ['mean', 'std']
}).round(2)
sinif_analiz

## 6. Korelasyon Analizi

In [ ]:
sayisal_kolonlar = ['FL_B', 'FL_A', 'VS', 'FL_B_3', 'FL_A_3', 'FL_B_12', 'FL_A_12']
korelasyon = df[sayisal_kolonlar].corr().round(3)
korelasyon

## 7. Gorsellestirme

In [ ]:
pivot1 = df.pivot_table(index='Container Type', values='FL_B', aggfunc='mean').reset_index()
pivot1 = pivot1.sort_values(by='FL_B', ascending=True)

plt.figure(figsize=(12, 7))
renkler = plt.cm.Greens(np.linspace(0.3, 0.9, len(pivot1)))
plt.barh(pivot1['Container Type'], pivot1['FL_B'], color=renkler)
plt.title('Konteyner Türlerine Göre Ortalama Doluluk Oranı', fontsize=14)
plt.xlabel('Ortalama FL_B Değeri')
plt.ylabel('Konteyner Türü')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
pivot2 = df.pivot_table(index='Recyclable fraction', values='FL_B', aggfunc='mean').reset_index()

plt.figure(figsize=(8, 8))
renkler2 = ['#3498db', '#e74c3c', '#2ecc71']
plt.pie(pivot2['FL_B'], labels=pivot2['Recyclable fraction'], autopct='%1.1f%%', 
        colors=renkler2, explode=[0.05, 0, 0], shadow=True, startangle=90)
plt.title('Atık Türlerinin Doluluk Oranı Dağılımı', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
pivot3 = df.pivot_table(index='Class', values=['FL_B', 'FL_A'], aggfunc='mean').reset_index()

x = np.arange(len(pivot3))
genislik = 0.35

fig, ax = plt.subplots(figsize=(9, 6))
bar1 = ax.bar(x - genislik/2, pivot3['FL_B'], genislik, label='FL_B (Sonra)', color='#9b59b6')
bar2 = ax.bar(x + genislik/2, pivot3['FL_A'], genislik, label='FL_A (Önce)', color='#f39c12')

ax.set_xlabel('Sınıf')
ax.set_ylabel('Doluluk Oranı')
ax.set_title('Sınıf Bazında Doluluk Karşılaştırması (FL_B vs FL_A)', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(pivot3['Class'])
ax.legend()
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(pivot_fl_b, annot=True, fmt='.1f', cmap='BuPu', linewidths=0.5, linecolor='white')
plt.title('Konteyner ve Atık Türü Kombinasyonlarının Doluluk Analizi', fontsize=14)
plt.xlabel('Geri Dönüşüm Türü')
plt.ylabel('Konteyner Tipi')
plt.tight_layout()
plt.show()

## 8. Sonuclar

Analiz sonucunda elde edilen bulgular:

In [ ]:
print("=" * 50)
print("SONUÇLAR")
print("=" * 50)
print(f"Toplam kayıt sayısı: {len(df)}")
print(f"Konteyner türü sayısı: {df['Container Type'].nunique()}")
print(f"En yüksek FL_B ortalaması: {pivot_fl_b.values.max():.2f}")
print(f"En düşük FL_B ortalaması: {pivot_fl_b.values.min():.2f}")

print("\nEn Yüksek Doluluk Kombinasyonları (FL_B > 71):")
for konteyner in pivot_fl_b.index:
    for atik in pivot_fl_b.columns:
        deger = pivot_fl_b.loc[konteyner, atik]
        if deger > 71:
            print(f"  {konteyner} + {atik}: {deger}")